In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm.notebook import tqdm
from scalecast.Forecaster import Forecaster, _sklearn_estimators_
from scalecast.auxmodels import mlp_stack, auto_arima

In [ ]:
mmap = {
    'Yearly':1,
    'Quarterly':4,
    'Monthly':12,
    'Weekly':1,
    'Daily':1,
    'Hourly':24,
}

high_thresh = 10
freqs = ('Yearly','Quarterly','Monthly','Weekly','Daily','Hourly')

In [ ]:
print('deleting bad models')
for freq in freqs:
    results = pickle.load(open(f'{freq}_results.pckl','rb'))
    # delete bad models
    for i, f in tqdm(results.items()):
        f.pop('avg','weighted')
        mpop = [
            m for m, r in f.history.items() if np.max(
                r['LevelForecast']
            ) > np.max(f.levely)*high_thresh
        ]
        if len(mpop) < len(f.history):
            f.pop(*mpop)
    with open(f'{freq}_results_final.pckl','wb') as pckl:
        pickle.dump(f,pckl)

In [ ]:
print('mlp stack modeling')
for freq in freqs:
    results = pickle.load(open(f'{freq}_results_final.pckl','rb'))
    for i, f in tqdm(results.items()):
        mlp_stack(
            f,
            models = [m for m in f.history if m in _sklearn_estimators_],
        )
    with open(f'{freq}_results_final.pckl','wb') as pckl:
        pickle.dump(f,pckl)

In [ ]:
print('auto arima forecasting')
for freq in freqs:
    results = pickle.load(open(f'{freq}_results_final.pckl','rb'))
    for i, f in tqdm(results.items()):
        f.undiff()
        auto_arima(
            f,
            m=mmap[freq] if mmap[freq] > 1 else None, 
            seasonal = mmap[freq] > 1,
            error_action='ignore',
            trace = False,
        )
    with open(f'{freq}_results_final.pckl','wb') as pckl:
        pickle.dump(f,pckl)

In [ ]:
print('theta forecasting')
for freq in freqs:
    results = pickle.load(open(f'{freq}_results_final.pckl','rb'))
    for i, f in tqdm(results.items()):
        f.set_estimator('theta')
        f.manual_forecast()
    with open(f'{freq}_results_final.pckl','wb') as pckl:
        pickle.dump(f,pckl)

In [ ]:
print('combo forecasting')
for freq in freqs:
    results = pickle.load(open(f'{freq}_results_final.pckl','rb'))
    for i, f in tqdm(results.items()):
        f.set_estimator('combo')
        for k in range(2,len(f.history) + 1):
            f.manual_forecast(
                models=f'top_{k}',
                determine_best_by='LevelTestSetMAPE',
                call_me=f'avg_top_{k}'
            )
    with open(f'{freq}_results_final.pckl','wb') as pckl:
        pickle.dump(f,pckl)